In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, Row, StringType


In [2]:
sc = SparkContext(master='local', appName='transformacionesyacciones')
spark2 = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/10 23:12:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path = '../curso/files/'

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark2)


/usr/local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Juego DF
PATH = '../curso/files/'
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('nombre_juego', StringType(), False),
    StructField('annio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

juegoDF = sqlContext.read.schema(juegoSchema)\
    .option("header", "true").csv(PATH+'juegos.csv')
juegoDF.show(5)

+--------+------------+-----+---------+---------+
|juego_id|nombre_juego|annio|temporada|   ciudad|
+--------+------------+-----+---------+---------+
|       1| 1896 Verano| 1896|   Verano|   Athina|
|       2| 1900 Verano| 1900|   Verano|    Paris|
|       3| 1904 Verano| 1904|   Verano|St. Louis|
|       4| 1906 Verano| 1906|   Verano|   Athina|
|       5| 1908 Verano| 1908|   Verano|   London|
+--------+------------+-----+---------+---------+
only showing top 5 rows



In [6]:
# deportista df
def eliminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

deportista_olimpicos_rdd = sc.textFile(path+'deportista.csv').map(lambda x: x.split(','))
deportista_olimpicos_rdd2 = sc.textFile(path+'deportista2.csv').map(lambda x: x.split(','))
deportista_olimpicos_rdd = deportista_olimpicos_rdd.union(deportista_olimpicos_rdd2)

deportista_olimpicos_rdd = deportista_olimpicos_rdd.mapPartitionsWithIndex(eliminar_encabezado)

deportista_olimpicos_rdd = deportista_olimpicos_rdd.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6]),
))

schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)    
])

deportistaDF = sqlContext.createDataFrame(deportista_olimpicos_rdd, schema)
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [7]:
# Deporte DF
deporteSchema = StructType([
    StructField('deporte_id', IntegerType(), False),
    StructField('deporte', StringType(), False),
])

deporteDF = sqlContext.read.schema(deporteSchema)\
    .option("header", "true").csv(PATH+'deporte.csv')
deporteDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [8]:
# evento DF
eventoSchema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('evento', StringType(), False),
    StructField('deporte_id', IntegerType(), False),
])

eventoDF = sqlContext.read.schema(eventoSchema)\
    .option("header", "true").csv(PATH+'evento.csv')
eventoDF.show(5)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



In [9]:
# juego DF
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('nombre_juego', StringType(), False),
    StructField('annio', IntegerType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False),
])

juegoDF = sqlContext.read.schema(juegoSchema) \
    .option("header", "true").csv(PATH+'juegos.csv')
juegoDF.show(5)

+--------+------------+-----+---------+---------+
|juego_id|nombre_juego|annio|temporada|   ciudad|
+--------+------------+-----+---------+---------+
|       1| 1896 Verano| 1896|   Verano|   Athina|
|       2| 1900 Verano| 1900|   Verano|    Paris|
|       3| 1904 Verano| 1904|   Verano|St. Louis|
|       4| 1906 Verano| 1906|   Verano|   Athina|
|       5| 1908 Verano| 1908|   Verano|   London|
+--------+------------+-----+---------+---------+
only showing top 5 rows



In [10]:
# juego DF
paisesSchema = StructType([
    StructField('id', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructField('sigla', StringType(), False),
])

paisesDF = sqlContext.read.schema(paisesSchema) \
    .option("header", "true").csv(PATH+'paises.csv')
paisesDF.show(5)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [11]:
!ls ../curso/files/
print('-----')
!head -n 5 ../curso/files/resultados.csv

Untitled.ipynb	deportista2.csv      juegos.csv		    resultados.csv
deporte.csv	deportistaError.csv  modelo_relacional.jpg
deportista.csv	evento.csv	     paises.csv
-----
resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [12]:
# resultados DF
resultadosSchema = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False),
])

resultadosDF = sqlContext.read.schema(resultadosSchema) \
    .option("header", "true").csv(PATH+'resultados.csv')
resultadosDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



### fin imports

In [13]:
deportistaDF = deportistaDF.withColumnRenamed('genero', 'sexo').drop('altura')

In [14]:
from pyspark.sql.functions import *
deportistaDF2 = deportistaDF.select(
    'deportista_id', 'nombre',
    col('edad').alias('edadAlJugar'),
    'equipo_id'
)

In [15]:
deportistaDF2.sort('edadAlJugar').show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|          133|           Franz Abb|          0|      399|
|          167|Ould Lamine Abdallah|          0|      362|
|           66|     Mohamed Abakkar|          0|     1003|
|          163|     Ismail Abdallah|          0|     1095|
|          139|George Ioannis Abbot|          0|     1043|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [16]:
deportistaDF2.filter(
    (deportistaDF2.edadAlJugar != 0)
).sort('edadAlJugar').show(3)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        47618|Sonja Henie Toppi...|         11|      742|
|        51268|      Beatrice Hutiu|         11|      861|
+-------------+--------------------+-----------+---------+
only showing top 3 rows



In [17]:
deportistaDF.join(
    resultadosDF,
    deportistaDF.deportista_id == resultadosDF.deportista_id,
    'left'
).join(
    juegoDF,
    juegoDF.juego_id == resultadosDF.juego_id,
    'left'
).join(
    eventoDF,
    eventoDF.evento_id == resultadosDF.evento_id,
    'left'
).select(
    deportistaDF.nombre, col('edad').alias('edad al jugar'),
    'medalla', col('annio').alias('año del juego'),
    eventoDF.evento.alias('Nombre de disciplina')
).show()

+--------------------+-------------+-------+-------------+--------------------+
|              nombre|edad al jugar|medalla|año del juego|Nombre de disciplina|
+--------------------+-------------+-------+-------------+--------------------+
|           A Dijiang|           24|     NA|         1992|Basketball Men's ...|
|            A Lamusi|           23|     NA|         2012|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|           24|     NA|         1920|Football Men's Fo...|
|Edgar Lindenau Aabye|           34|   Gold|         1900|Tug-Of-War Men's ...|
|Christine Jacoba ...|           21|     NA|         1994|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|         1994|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|         1992|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|         1992|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|         1988|Speed Skating Wom...|
|Christine Jacoba ...|           21|    

todas las medallas ganadoras unidas con el pais y equipo 

In [18]:
resultadosDF.filter(resultadosDF.medalla != 'NA') \
    .join(deportistaDF, deportistaDF.deportista_id == resultadosDF.deportista_id, 'left') \
    .join(paisesDF, paisesDF.id == deportistaDF.equipo_id, 'left') \
    .select(
        deportistaDF.nombre, col('edad').alias('edad al jugar'),
        'medalla',
        paisesDF.equipo,
        paisesDF.sigla
    ).sort(col('sigla')).show(5)


+-----------------+-------------+-------+--------------------+-----+
|           nombre|edad al jugar|medalla|              equipo|sigla|
+-----------------+-------------+-------+--------------------+-----+
|             null|         null| Silver|                null| null|
|  Rohullah Nikpai|           21| Bronze|         Afghanistan|  AFG|
|  Rohullah Nikpai|           21| Bronze|         Afghanistan|  AFG|
|   Jan D. Boersma|           19| Silver|Netherlands Antilles|  AHO|
|Majlinda Kelmendi|           21|   Gold|             Albania|  ALB|
+-----------------+-------------+-------+--------------------+-----+
only showing top 5 rows



In [19]:
paisesDF.printSchema()

root
 |-- id: integer (nullable = true)
 |-- equipo: string (nullable = true)
 |-- sigla: string (nullable = true)



In [21]:
medallistaXAnio = deportistaDF.join(
        resultadoDF, 
        deportistaDF.deportista_id == resultadoDF.deportista_id, 
        "left"
    ).join(
        juegoDF, 
        juegoDF.juego_id == resultadoDF.juego_id, 
        "left"
    ).join(
        paisesDF, 
        deportistaOlimpicoDF.equipo_id == paisesDF.id, 
        "left"
    ).join(
        deportesOlimpicosDF, 
        deportesOlimpicosDF.evento_id == resultadoDF.evento_id, 
        "left"
    ).join(
        deportesDF, 
        deportesOlimpicosDF.deporte_id == deportesDF.deporte_id, 
        "left"
    ).select(
        "sigla",
        "anio",
        "medalla",
        deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
        deportesDF.deporte.alias("Nombre disciplina"),
        deportistaOlimpicoDF.nombre    
    )



NameError: name 'deportistaOlimpicoDF' is not defined

In [ ]:
medallistaXAnio.show()

In [20]:
# spark2.stop()